In [ ]:
'''
https://ileanacabada.medium.com/price-elasticity-of-demand-using-linear-regression-in-python-part-1-a28c844c1656
https://medium.com/geekculture/price-elasticity-of-demand-using-linear-regression-in-python-part-2-8adb654328e7
'''

'\nhttps://ileanacabada.medium.com/price-elasticity-of-demand-using-linear-regression-in-python-part-1-a28c844c1656\nhttps://medium.com/geekculture/price-elasticity-of-demand-using-linear-regression-in-python-part-2-8adb654328e7\n'

In [ ]:
'''
level level
---------------------
q = c + m * p
e = (m * p) / q

log level
---------------------
log(q) = c + m * p
e = m * p

level log
---------------------
q = c + m * log(x)
e = m / q

log log
---------------------
log(q) = c + m * log(p)
e = m
'''

'\nlevel level\n---------------------\nq = c + m * p\ne = (m * p) / q\n\nlog level\n---------------------\nlog(q) = c + m * p\ne = m * p\n\nlevel log\n---------------------\nq = c + m * log(x)\ne = m / q\n\nlog log\n---------------------\nlog(q) = c + m * log(p)\ne = m\n'

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pyplot as plt
import math
from statsmodels.formula.api import ols

In [ ]:
br1_ctg = [142, 18, 131, 166, 22, 25, 233, 1, 196, 54, 4, 58]

In [ ]:
elasticity_explanation = ['self-inelastic', 'self-unit-elastic', 'self-elastic', 'self-positive-elastic']    

In [ ]:
def divergent_plot(df, values_column, ylabel, xlabel):

    #Divergent plot
    df['overall_ped_ranking'] = df[values_column].rank( ascending = True).astype(int)
    df.sort_values(values_column, ascending =False, inplace = True)
    plt.figure(figsize = (10,50), dpi = 100)
    plt.hlines(y = df['overall_ped_ranking'] , xmin = 0, xmax = df[values_column], alpha = 0.5, linewidth = 3)
    
    #Add elasticity labels
    for x, y, tex in zip(df[values_column], df['overall_ped_ranking'] , df[values_column]):
        plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left', 
                 verticalalignment='center', fontdict={'color':'red' if x < 0 else 'green', 'size':10})
        
    
    # Axis and title
    plt.gca().set(ylabel= ylabel, xlabel= xlabel)
    plt.yticks(df['overall_ped_ranking'])
    plt.title(values_column , fontdict={'size':15})
    plt.grid(linestyle='--', alpha=0.5)
    plt.show()
            
    
    #Adjust Ranking column and print dataframe
    pd.set_option('display.width', 4000)
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    df = df.iloc[:,:3]
    df.set_index('overall_ped_ranking', inplace=True)
    display(df)

In [ ]:
df_ctg_ppg = pd.read_csv("prod_ctg_ppg.csv")
df_ctg_ppg.columns = [c.lower() for c in df_ctg_ppg.columns]
df_ctg_ppg = df_ctg_ppg.dropna()
df_ctg_ppg.prod_category = df_ctg_ppg['prod_category'].astype(int)
df_ctg = df_ctg_ppg[['prod_category', 'category_desc']].drop_duplicates()
df_ppg_list = df_ctg_ppg[['ppg_id', 'ppg_desc']].drop_duplicates()
df_ctg_ppg

In [ ]:
df_uch_level_g = pd.read_csv("result_uch_level_e_uch_level_g.csv")
df_uch_level_g.columns = [c.lower() for c in df_uch_level_g.columns]
df_uch_level_g = df_uch_level_g.dropna()
df_uch_level_g

In [ ]:
#Format and build a dataframe with x_values for each product within the uch_level_g 
df = pd.read_csv("result_elasticity_data_ctg.csv")
df.columns = [c.lower() for c in df.columns]
df['total_sales_units_log'] = np.log(df.total_sales_units)
df['avg_price_per_unit_to_consmer_log'] = np.log(df.avg_price_per_unit_to_consmer)
orig_df = df.copy()
df = df[df.prod_category.isin(br1_ctg)]
df = df.sort_values(by=['fscl_wk_end_dt', 'prod_category', 'uch_level_g'])

unique_cust_ctg_comb_list = df[['uch_level_g', 'prod_category']].drop_duplicates().values.tolist()

df_unique_cust_ctg_comb = pd.DataFrame(df.groupby(['uch_level_g', 'prod_category']).size()).reset_index()
df_unique_cust_ctg_comb.columns = [ 'uch_level_g', 'prod_category', 'combination_count']
error_list = []

ols_results_values = {
    "uch_level_g": [],
    "prod_category": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
    "msg" : []
}

for uc in unique_cust_ctg_comb_list:
    #output_str = "Record count for the combination " + uc[0] + ", prod_category " + str(uc[1]) + " is " + df_unique_cust_ctg_comb[(df_unique_cust_ctg_comb.uch_level_g ==  uc[0]) & (df_unique_cust_ctg_comb.prod_category == uc[1])]['combination_count'].astype(str)
    print("customer " + uc[0])
    print("prod_category " + str(uc[1]))
    print("Combination count " + str(df_unique_cust_ctg_comb[(df_unique_cust_ctg_comb.uch_level_g ==  uc[0]) & (df_unique_cust_ctg_comb.prod_category == uc[1])][['combination_count']].values[0][0]))
    print("******")
    test_df = df[(df.uch_level_g == uc[0]) & (df.prod_category == uc[1])][['fscl_wk_end_dt', 'total_sales_units_log', 'avg_price_per_unit_to_consmer_log']]
    test_df.set_index("fscl_wk_end_dt", inplace=True)
    try:
        model_cust_ctg_elasticity = ols("total_sales_units_log ~ avg_price_per_unit_to_consmer_log", data = test_df).fit()
        print(model_cust_ctg_elasticity.summary())
        
        rsquared = model_cust_ctg_elasticity.rsquared
        coefficient_pvalue = model_cust_ctg_elasticity.f_pvalue
        intercept, slope = model_cust_ctg_elasticity.params
        mean_price = np.mean(test_df.avg_price_per_unit_to_consmer_log)
        mean_quantity = np.mean(test_df.total_sales_units_log)
        tintercept, t_score = model_cust_ctg_elasticity.tvalues
     
        #Price elasticity Formula
        #price_elasticity = (slope)*(mean_price/mean_quantity)    
        price_elasticity = (slope)
            
        #Append results into dictionary for dataframe
        ols_results_values["uch_level_g"].append(uc[0])
        ols_results_values["prod_category"].append(uc[1])
        ols_results_values["price_elasticity"].append(price_elasticity)
        ols_results_values["price_mean"].append(mean_price)
        ols_results_values["quantity_mean"].append(mean_quantity)
        ols_results_values["intercept"].append(intercept)
        ols_results_values['t_score'].append(t_score)
        ols_results_values["slope"].append(slope)
        ols_results_values["coefficient_pvalue"].append(coefficient_pvalue)
        ols_results_values['rsquared'].append(rsquared)
        if coefficient_pvalue < 0.05:
            ols_results_values["msg"].append("coefficient_pvalue < 0.05, statistically significant results")
        else:
            ols_results_values["msg"].append("coefficient_pvalue >= 0.05, statistically insignificant results")
        print("Model results printed")
    except:
        print("Error occurred")
        error_list.append(uc)

final_df = pd.DataFrame.from_dict(ols_results_values)
df_elasticity = final_df[['uch_level_g', 'prod_category', 'price_elasticity', 't_score', 'coefficient_pvalue', 'slope', 'price_mean', 'quantity_mean', 'intercept', 'rsquared', 'msg']]

pe_plot = divergent_plot(df_elasticity, 'price_elasticity', 'overall_ped_ranking', 'price_elasticity')
df_elasticity = df_elasticity.sort_values(by=['prod_category', 'price_elasticity'], ascending=[True, True])

df_elasticity_l1 = pd.merge(df_elasticity, df_ctg, on = ['prod_category'], how='inner')
df_elasticity_l2 = pd.merge(df_elasticity_l1, df_uch_level_g, on = ['uch_level_g'], how='inner')

cust_ctg_elasticity_conditions_1 = [
    (df_elasticity_l2['price_elasticity'] <= 0) & (df_elasticity_l2['price_elasticity'] > -1),
    (df_elasticity_l2['price_elasticity'] == -1.000000000),
    (df_elasticity_l2['price_elasticity'] < -1),
    (df_elasticity_l2['price_elasticity'] > 0)
    ]    

df_elasticity_l2['elasticity_explanation'] = np.select(cust_ctg_elasticity_conditions_1, elasticity_explanation)
df_elasticity_l2.to_csv("canada_cust_ctg_elasticity_details.csv", index=False)

df_ped = pd.DataFrame()
for ctg in sorted(list(df_elasticity['prod_category'].drop_duplicates())):
    test_df_ped = df_elasticity[(df_elasticity.prod_category == ctg)].reset_index().iloc[:,1:]
    test_df_ped['ctg_ped_ranking'] = test_df_ped['price_elasticity'].rank(ascending = True).astype(int)
    test_df_ped.sort_values('price_elasticity', ascending =False, inplace = True)
    plt.figure(figsize = (20,10), dpi = 100)
    plt.hlines(y = test_df_ped['ctg_ped_ranking'] , xmin = 0, xmax = test_df_ped['price_elasticity'], alpha = 0.5, linewidth = 3)
    
    
    #Add elasticity labels
    for x, y, tex in zip(test_df_ped['price_elasticity'], test_df_ped['ctg_ped_ranking'] , test_df_ped['price_elasticity']):
        plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left', 
                 verticalalignment='center', fontdict={'color':'red' if x < 0 else 'green', 'size':15})
    
    # Axis and title
    ylabel, xlabel = 'Ranking Number', 'Price Elasticity'
    plt.gca().set(ylabel= ylabel, xlabel= xlabel)
    plt.yticks(test_df_ped['ctg_ped_ranking'])
    plt.title('price_elasticity' , fontdict={'size':15})
    plt.grid(linestyle='--', alpha=0.5)
    plt.show()
    
    #Adjust Ranking column and print dataframe
    pd.set_option('display.width', 4000)
    #cols = list(test_df_ped.columns)
    #cols = [cols[-1]] + cols[:3]
    cols = ['uch_level_g', 'prod_category', 'price_elasticity', 'ctg_ped_ranking', 'msg']
    test_df_ped = test_df_ped[cols]
    
    test_df_ped = test_df_ped.iloc[:,:5]
    test_df_ped.set_index('ctg_ped_ranking', inplace=True)
    display(test_df_ped)
    test_df_ped['ctg_ped_ranking'] = test_df_ped.index
    df_ped = df_ped.append(test_df_ped)

df_ped = df_ped.sort_values(by=['prod_category', 'price_elasticity'], ascending=[True, True])
cust_ctg_elasticity_conditions = [
    (df_ped['price_elasticity'] <= 0) & (df_ped['price_elasticity'] > -1),
    (df_ped['price_elasticity'] == -1.000000000),
    (df_ped['price_elasticity'] < -1),
    (df_ped['price_elasticity'] > 0)
    ]
df_ped['elasticity_explanation'] = np.select(cust_ctg_elasticity_conditions, elasticity_explanation)

#df_ped.to_csv("canada_cust_ctg_elasticity_summary.csv", index=False)
df_ped_l1 = pd.merge(df_ped, df_ctg, on = ['prod_category'], how='inner')
df_ped_l2 = pd.merge(df_ped_l1, df_uch_level_g, on = ['uch_level_g'], how='inner')
df_ped_l2.to_csv("canada_cust_ctg_elasticity_summary.csv", index=False)


In [ ]:
orig_df_pivot = orig_df.pivot(index = 'fscl_wk_end_dt', columns = ['uch_level_g','prod_category'], values = 'total_sales_units')
formatted_column_name_list = []
for c in orig_df_pivot.columns:
    formatted_column_name_list.append(c[0] + '_ctg' + str(c[1]))
orig_df_pivot.columns = formatted_column_name_list
orig_df_pivot

In [ ]:
orig_df_pivot.plot(figsize=(40,20))

In [ ]:
file_name_1 = "result_elasticity_data_ppg_1.csv"
file_name_2 = "result_elasticity_data_ppg_2.csv"

df1 = pd.read_csv(file_name_1)
df2 = pd.read_csv(file_name_2)
df_ppg = pd.concat([df1, df2])
df_ppg.columns = [c.lower() for c in df_ppg.columns]
df_ppg['total_sales_units_log'] = np.log(df_ppg.total_sales_units)
df_ppg['avg_price_per_unit_to_consmer_log'] = np.log(df_ppg.avg_price_per_unit_to_consmer)
orig_df_ppg = df_ppg.copy()
orig_df_ppg.columns = [c.lower() for c in orig_df_ppg.columns]
df_ppg = df_ppg[df_ppg.prod_category.isin(br1_ctg)]
df_ppg = df_ppg.sort_values(by=['fscl_wk_end_dt', 'prod_category', 'ppg_id', 'uch_level_g'])

unique_cust_ctg_ppg_comb_list = df_ppg[['uch_level_g', 'prod_category', 'ppg_id']].drop_duplicates().values.tolist()

df_unique_cust_ctg_ppg_comb = pd.DataFrame(df_ppg.groupby(['uch_level_g', 'prod_category', 'ppg_id']).size()).reset_index()
df_unique_cust_ctg_ppg_comb.columns = [ 'uch_level_g', 'prod_category', 'ppg_id', 'combination_count']
error_list = []

ols_results_values_1 = {
    "uch_level_g": [],
    "prod_category": [],
	  "ppg_id": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
    "msg" : []
}

for uc in unique_cust_ctg_ppg_comb_list:
    print("customer " + uc[0])
    print("prod_category " + str(uc[1]))
    print("ppg_id " + str(uc[2]))
    val = (df_unique_cust_ctg_ppg_comb[(df_unique_cust_ctg_ppg_comb.uch_level_g ==  uc[0]) & 
                                      (df_unique_cust_ctg_ppg_comb.prod_category == uc[1]) & 
                                      (df_unique_cust_ctg_ppg_comb.ppg_id == uc[2])][['combination_count']].values[0][0])
    print("Combination count " + str(val))
    print("******")
    test_df_ppg = df_ppg[(df_ppg.uch_level_g == uc[0]) & (df_ppg.prod_category == uc[1]) & (df_ppg.ppg_id == uc[2])][['fscl_wk_end_dt', 'total_sales_units_log', 'avg_price_per_unit_to_consmer_log']]
    test_df_ppg.set_index("fscl_wk_end_dt", inplace=True)
    try:
        model_cust_ctg_ppg_elasticity = ols("total_sales_units_log ~ avg_price_per_unit_to_consmer_log", data = test_df_ppg).fit()
        print(model_cust_ctg_ppg_elasticity.summary())
        
        rsquared = model_cust_ctg_ppg_elasticity.rsquared
        coefficient_pvalue = model_cust_ctg_ppg_elasticity.f_pvalue
        intercept, slope = model_cust_ctg_ppg_elasticity.params
        mean_price = np.mean(test_df_ppg.avg_price_per_unit_to_consmer_log)
        mean_quantity = np.mean(test_df_ppg.total_sales_units_log)
        tintercept, t_score = model_cust_ctg_ppg_elasticity.tvalues
     
        #Price elasticity Formula
        #price_elasticity = (slope)*(mean_price/mean_quantity)    
        price_elasticity = (slope)
            
        #Append results into dictionary for dataframe
        ols_results_values_1["uch_level_g"].append(uc[0])
        ols_results_values_1["prod_category"].append(uc[1])
        ols_results_values_1["ppg_id"].append(uc[2])
        ols_results_values_1["price_elasticity"].append(price_elasticity)
        ols_results_values_1["price_mean"].append(mean_price)
        ols_results_values_1["quantity_mean"].append(mean_quantity)
        ols_results_values_1["intercept"].append(intercept)
        ols_results_values_1['t_score'].append(t_score)
        ols_results_values_1["slope"].append(slope)
        ols_results_values_1["coefficient_pvalue"].append(coefficient_pvalue)
        ols_results_values_1['rsquared'].append(rsquared)
        if coefficient_pvalue < 0.05:
            ols_results_values_1["msg"].append("coefficient_pvalue < 0.05, statistically significant results")
        else:
            ols_results_values_1["msg"].append("coefficient_pvalue >= 0.05, statistically insignificant results")
        print("Model results printed")
    except:
        print("Error occurred")
        error_list.append(uc)

final_df_ppg = pd.DataFrame.from_dict(ols_results_values_1)
df_elasticity_ppg = final_df_ppg[['uch_level_g', 'prod_category', 'ppg_id', 'price_elasticity', 't_score', 'coefficient_pvalue', 'slope', 'price_mean', 'quantity_mean', 'intercept', 'rsquared', 'msg']]

pe_plot_ppg = divergent_plot(df_elasticity_ppg, 'price_elasticity', 'overall_ped_ranking', 'price_elasticity')
df_elasticity_ppg = df_elasticity_ppg.sort_values(by=['prod_category', 'ppg_id', 'price_elasticity'], ascending=[True, True, True])

df_elasticity_ppg_l1 = pd.merge(df_elasticity_ppg, df_ctg_ppg, on = ['prod_category', 'ppg_id'], how='inner')
df_elasticity_ppg_l2 = pd.merge(df_elasticity_ppg_l1, df_uch_level_g, on = ['uch_level_g'], how='inner')
cust_ctg_ppg_elasticity_conditions_1 = [
    (df_elasticity_ppg_l2['price_elasticity'] <= 0) & (df_elasticity_ppg_l2['price_elasticity'] > -1),
    (df_elasticity_ppg_l2['price_elasticity'] == -1.000000000),
    (df_elasticity_ppg_l2['price_elasticity'] < -1),
    (df_elasticity_ppg_l2['price_elasticity'] > 0)
    ]       
df_elasticity_ppg_l2['elasticity_explanation'] = np.select(cust_ctg_ppg_elasticity_conditions_1, elasticity_explanation)
df_elasticity_ppg_l2.to_csv("canada_cust_ctg_ppg_elasticity_details.csv", index=False)

df_ped_ppg = pd.DataFrame()
for i in df_elasticity_ppg[['prod_category', 'ppg_id']].drop_duplicates().values.tolist():
    test_df_ppg_ped = df_elasticity_ppg[(df_elasticity_ppg.prod_category == i[0]) & (df_elasticity_ppg.ppg_id == i[1])].reset_index().iloc[:,1:]
    test_df_ppg_ped['ctg_ppg_ped_ranking'] = test_df_ppg_ped['price_elasticity'].rank(ascending = True).astype(int)
    test_df_ppg_ped.sort_values('price_elasticity', ascending =False, inplace = True)
    plt.figure(figsize = (20,10), dpi = 100)
    plt.hlines(y = test_df_ppg_ped['ctg_ppg_ped_ranking'] , xmin = 0, xmax = test_df_ppg_ped['price_elasticity'], alpha = 0.5, linewidth = 3)
    
    
    #Add elasticity labels
    for x, y, tex in zip(test_df_ppg_ped['price_elasticity'], test_df_ppg_ped['ctg_ppg_ped_ranking'] , test_df_ppg_ped['price_elasticity']):
        plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left', 
                 verticalalignment='center', fontdict={'color':'red' if x < 0 else 'green', 'size':15})
    
    # Axis and title
    ylabel, xlabel = 'Ranking Number', 'Price Elasticity'
    plt.gca().set(ylabel= ylabel, xlabel= xlabel)
    plt.yticks(test_df_ppg_ped['ctg_ppg_ped_ranking'])
    plt.title('price_elasticity' , fontdict={'size':15})
    plt.grid(linestyle='--', alpha=0.5)
    plt.show()
    
    #Adjust Ranking column and print dataframe
    pd.set_option('display.width', 4000)
    #cols = list(test_df_ppg_ped.columns)
    #cols = [cols[-1]] + cols[:4]
    cols = ['uch_level_g', 'prod_category', 'ppg_id', 'price_elasticity', 'ctg_ppg_ped_ranking', 'msg']
    test_df_ppg_ped = test_df_ppg_ped[cols]
    
    test_df_ppg_ped = test_df_ppg_ped.iloc[:,:6]
    test_df_ppg_ped.set_index('ctg_ppg_ped_ranking', inplace=True)
    display(test_df_ppg_ped)
    test_df_ppg_ped['ctg_ppg_ped_ranking'] = test_df_ppg_ped.index
    df_ped_ppg = df_ped_ppg.append(test_df_ppg_ped)

df_ped_ppg = df_ped_ppg.sort_values(by=['prod_category', 'ppg_id', 'price_elasticity'], ascending=[True, True, True])
cust_ctg_ppg_elasticity_conditions = [
    (df_ped_ppg['price_elasticity'] <= 0) & (df_ped_ppg['price_elasticity'] > -1),
    (df_ped_ppg['price_elasticity'] == -1.000000000),
    (df_ped_ppg['price_elasticity'] <= -1),
    (df_ped_ppg['price_elasticity'] > 0)
    ]   
df_ped_ppg['elasticity_explanation'] = np.select(cust_ctg_ppg_elasticity_conditions, elasticity_explanation)

#df_ped_ppg.to_csv("canada_cust_ctg_elasticity_summary.csv", index=False)
df_ped_ppg_l1 = pd.merge(df_ped_ppg, df_ctg_ppg, on = ['prod_category', 'ppg_id'], how='inner')
df_ped_ppg_l2 = pd.merge(df_ped_ppg_l1, df_uch_level_g, on = ['uch_level_g'], how='inner')
df_ped_ppg_l2.to_csv("canada_cust_ctg_ppg_elasticity_summary.csv", index=False)


In [ ]:
df_elasticity_ppg_l2[(df_elasticity_ppg_l2.price_elasticity == 0) & (df_elasticity_ppg_l2.ppg_id=='025132003396Y46')]

,uch_level_g,prod_category,ppg_id,price_elasticity,t_score,coefficient_pvalue,slope,price_mean,quantity_mean,intercept,rsquared,msg,overall_ped_ranking,category_desc,ppg_desc,uch_level_e,uch_level_e_desc,uch_level_g_desc,elasticity_explanation


In [ ]:
df_ped_ppg_l2[(df_ped_ppg_l2.price_elasticity == 0) & (df_ped_ppg_l2.ppg_id=='142047001404CG6')]

In [ ]:
df_elasticity_ppg_l2[(df_elasticity_ppg_l2.ppg_id=='018078502211VI2') & (df_elasticity_ppg_l2.uch_level_g=='CA4000428')][['elasticity_explanation', 'price_elasticity']]

In [ ]:
df_ntl_ctg = pd.read_csv("result_national_elasticity_category.csv")
df_ntl_ctg.columns = [c.lower() for c in df_ntl_ctg.columns]
df_ntl_ctg['total_sales_units_log'] = np.log(df_ntl_ctg.total_sales_units)
df_ntl_ctg['avg_price_per_unit_to_consmer_log'] = np.log(df_ntl_ctg.avg_price_per_unit_to_consmer)
orig_df_ntl_ctg = df_ntl_ctg.copy()
df_ntl_ctg = df_ntl_ctg[df_ntl_ctg.prod_category.isin(br1_ctg)]
df_ntl_ctg = df_ntl_ctg.sort_values(by=['fscl_wk_end_dt', 'prod_category'])

unique_ctg_list = sorted(list(df_ntl_ctg['prod_category'].drop_duplicates()))

ntl_ctg_results_values = {
    "prod_category": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
    "msg" : []
}
ntl_err_list = []
for c in unique_ctg_list:
    print("Processing prod category " + str(c))
    test_ntl_ctg_df = df[(df.prod_category == c)][['fscl_wk_end_dt', 'total_sales_units_log', 'avg_price_per_unit_to_consmer_log']]
    test_ntl_ctg_df.set_index("fscl_wk_end_dt", inplace=True)
    try:
        model_ntl_ctg = ols("total_sales_units_log ~ avg_price_per_unit_to_consmer_log", data = test_ntl_ctg_df).fit()
        print(model_ntl_ctg.summary())
        
        rsquared = model_ntl_ctg.rsquared
        print("rsquared " + str(rsquared))
        coefficient_pvalue = model_ntl_ctg.f_pvalue
        print("coefficient_pvalue " + str(coefficient_pvalue))
        intercept, slope = model_ntl_ctg.params
        print("intercept " + str(intercept))
        print("slope " + str(slope))
        mean_price = np.mean(df_ntl_ctg[df_ntl_ctg.prod_category == c].avg_price_per_unit_to_consmer_log)
        print("mean_price " + str(mean_price))
        mean_quantity = np.mean(df_ntl_ctg[df_ntl_ctg.prod_category == c].total_sales_units_log)
        print("mean_quantity " + str(mean_quantity))
        tintercept, t_score = model_ntl_ctg.tvalues
        print("tintercept " + str(tintercept))
        print("t_score " + str(t_score))
        
        price_elasticity = (slope)
        
        #Append results into dictionary for dataframe
        ntl_ctg_results_values["prod_category"].append(c)
        ntl_ctg_results_values["price_elasticity"].append(price_elasticity)
        ntl_ctg_results_values["price_mean"].append(mean_price)
        ntl_ctg_results_values["quantity_mean"].append(mean_quantity)
        ntl_ctg_results_values["intercept"].append(intercept)
        ntl_ctg_results_values['t_score'].append(t_score)
        ntl_ctg_results_values["slope"].append(slope)
        ntl_ctg_results_values["coefficient_pvalue"].append(coefficient_pvalue)
        ntl_ctg_results_values['rsquared'].append(rsquared)
        if coefficient_pvalue < 0.05:
            ntl_ctg_results_values["msg"].append("coefficient_pvalue < 0.05, statistically significant results")
        else:
            ntl_ctg_results_values["msg"].append("coefficient_pvalue >= 0.05, statistically insignificant results")
        print("Model results printed")
        print("*******************************************************************")
    except:
        print("Error occurred")
        ntl_err_list.append(c)

final_df_ntl_ctg = pd.DataFrame.from_dict(ntl_ctg_results_values)
df_ntl_ctg_elasticity = final_df_ntl_ctg[['prod_category','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared','msg']]
pe_plot = divergent_plot(df_ntl_ctg_elasticity, 'price_elasticity', 'overall_ped_ranking', 'price_elasticity')

df_ntl_ctg_elasticity_l1 = pd.merge(df_ntl_ctg_elasticity, df_ctg, on = ['prod_category'], how='inner')
ntl_ctg_elasticity_conditions = [
    (df_ntl_ctg_elasticity_l1['price_elasticity'] <= 0) & (df_ntl_ctg_elasticity_l1['price_elasticity'] > -1),
    (df_ntl_ctg_elasticity_l1['price_elasticity'] == -1.000000000),
    (df_ntl_ctg_elasticity_l1['price_elasticity'] < -1),
    (df_ntl_ctg_elasticity_l1['price_elasticity'] > 0)
    ]  
df_ntl_ctg_elasticity_l1['elasticity_explanation'] = np.select(ntl_ctg_elasticity_conditions, elasticity_explanation)
df_ntl_ctg_elasticity_l1.to_csv("canada_national_category_elasticity_details.csv", index=False)
df_ntl_ctg_elasticity_l1[['prod_category', 'category_desc', 'price_elasticity', 'overall_ped_ranking', 'msg', 'elasticity_explanation']].sort_values(by = ['price_elasticity']).to_csv("canada_national_category_elasticity_summary.csv", index=False)

In [ ]:
orig_df_ntl_ctg_pivot = df_ntl_ctg.pivot(index = 'fscl_wk_end_dt', columns = ['prod_category'], values = 'total_sales_units')
formatted_column_name_list = []
for c in orig_df_ntl_ctg_pivot.columns:
    formatted_column_name_list.append('ctg_' + str(c))
orig_df_ntl_ctg_pivot.columns = formatted_column_name_list
orig_df_ntl_ctg_pivot

In [ ]:
for c in df_ntl_ctg.prod_category.values.tolist():
    print("Processing category " + str(c))
    plt.show( block=False )
    temp_ntl_df = df_ntl_ctg[df_ntl_ctg.prod_category ==  c]
    temp_ntl_df.set_index('fscl_wk_end_dt', inplace = True)
    temp_ntl_df[temp_ntl_df.prod_category ==  c].total_sales_units.plot()

In [ ]:
orig_df_ntl_ctg_pivot.plot(figsize=(20,10))

In [ ]:
df_ntl_cust = df.groupby(['uch_level_g', 'fscl_wk_end_dt']).agg({"total_sales_units_log" : "sum", "avg_price_per_unit_to_consmer_log" : "sum"}).reset_index()
df_ntl_cust = df_ntl_cust.sort_values(by=['fscl_wk_end_dt', 'uch_level_g'])

unique_ctg_list = sorted(list(df_ntl_cust['uch_level_g'].drop_duplicates()))

ntl_cust_results_values = {
    "uch_level_g": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
    "msg" : []
}
ntl_err_list = []
for c in unique_ctg_list:
    print("Processing prod category " + str(c))
    test_ntl_ctg_df = df[(df.uch_level_g == c)][['fscl_wk_end_dt', 'total_sales_units_log', 'avg_price_per_unit_to_consmer_log']]
    test_ntl_ctg_df.set_index("fscl_wk_end_dt", inplace=True)
    try:
        model_ntl_ctg = ols("total_sales_units_log ~ avg_price_per_unit_to_consmer_log", data = test_ntl_ctg_df).fit()
        print(model_ntl_ctg.summary())
        
        rsquared = model_ntl_ctg.rsquared
        print("rsquared " + str(rsquared))
        coefficient_pvalue = model_ntl_ctg.f_pvalue
        print("coefficient_pvalue " + str(coefficient_pvalue))
        intercept, slope = model_ntl_ctg.params
        print("intercept " + str(intercept))
        print("slope " + str(slope))
        mean_price = np.mean(df_ntl_cust[df_ntl_cust.uch_level_g == c].avg_price_per_unit_to_consmer_log)
        print("mean_price " + str(mean_price))
        mean_quantity = np.mean(df_ntl_cust[df_ntl_cust.uch_level_g == c].total_sales_units_log)
        print("mean_quantity " + str(mean_quantity))
        tintercept, t_score = model_ntl_ctg.tvalues
        print("tintercept " + str(tintercept))
        print("t_score " + str(t_score))
        
        price_elasticity = (slope)
        
        #Append results into dictionary for dataframe
        ntl_cust_results_values["uch_level_g"].append(c)
        ntl_cust_results_values["price_elasticity"].append(price_elasticity)
        ntl_cust_results_values["price_mean"].append(mean_price)
        ntl_cust_results_values["quantity_mean"].append(mean_quantity)
        ntl_cust_results_values["intercept"].append(intercept)
        ntl_cust_results_values['t_score'].append(t_score)
        ntl_cust_results_values["slope"].append(slope)
        ntl_cust_results_values["coefficient_pvalue"].append(coefficient_pvalue)
        ntl_cust_results_values['rsquared'].append(rsquared)
        if coefficient_pvalue < 0.05:
            ntl_cust_results_values["msg"].append("coefficient_pvalue < 0.05, statistically significant results")
        else:
            ntl_cust_results_values["msg"].append("coefficient_pvalue >= 0.05, statistically insignificant results")
        print("Model results printed")
        print("*******************************************************************")
    except:
        print("Error occurred")
        ntl_err_list.append(c)

final_df_ntl_cust = pd.DataFrame.from_dict(ntl_cust_results_values)
df_ntl_cust_elasticity = final_df_ntl_cust[['uch_level_g','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared','msg']]
pe_plot = divergent_plot(df_ntl_cust_elasticity, 'price_elasticity', 'overall_ped_ranking', 'price_elasticity')

df_ntl_cust_elasticity_l1 = pd.merge(df_ntl_cust_elasticity, df_uch_level_g, on = ['uch_level_g'], how='inner')
ntl_cust_elasticity_conditions = [
    (df_ntl_cust_elasticity_l1['price_elasticity'] <= 0) & (df_ntl_cust_elasticity_l1['price_elasticity'] > -1),
    (df_ntl_cust_elasticity_l1['price_elasticity'] == -1.000000000),
    (df_ntl_cust_elasticity_l1['price_elasticity'] < -1),
    (df_ntl_cust_elasticity_l1['price_elasticity'] > 0)
    ]   
df_ntl_cust_elasticity_l1['elasticity_explanation'] = np.select(ntl_cust_elasticity_conditions, elasticity_explanation)
df_ntl_cust_elasticity_l1.to_csv("canada_national_customer_elasticity_details.csv", index=False)
df_ntl_cust_elasticity_l1[['uch_level_e', 'uch_level_e_desc','uch_level_g', 'uch_level_g_desc', 'price_elasticity', 'overall_ped_ranking', 'msg', 'elasticity_explanation']].sort_values(by = ['price_elasticity']).to_csv("canada_national_customer_elasticity_summary.csv", index=False)

In [ ]:
df_ntl_ppg = df_ppg.groupby(['ppg_id', 'fscl_wk_end_dt']).agg({"total_sales_units_log" : "sum", "avg_price_per_unit_to_consmer_log" : "sum"}).reset_index()
df_ntl_ppg = df_ntl_ppg.sort_values(by=['fscl_wk_end_dt', 'ppg_id'])

unique_ppg_list = sorted(list(df_ntl_ppg['ppg_id'].drop_duplicates()))

ntl_ppg_results_values = {
    "ppg_id": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared" : [],
    "msg" : []
}
ntl_err_list = []
for c in unique_ppg_list:
    print("Processing ppg " + c)
    test_ntl_ppg_df = df[(df.uch_level_g == c)][['fscl_wk_end_dt', 'total_sales_units_log', 'avg_price_per_unit_to_consmer_log']]
    test_ntl_ppg_df.set_index("fscl_wk_end_dt", inplace=True)
    try:
        model_ntl_ppg = ols("total_sales_units_log ~ avg_price_per_unit_to_consmer_log", data = test_ntl_ppg_df).fit()
        print(model_ntl_ppg.summary())
        
        rsquared = model_ntl_ppg.rsquared
        print("rsquared " + str(rsquared))
        coefficient_pvalue = model_ntl_ppg.f_pvalue
        print("coefficient_pvalue " + str(coefficient_pvalue))
        intercept, slope = model_ntl_ppg.params
        print("intercept " + str(intercept))
        print("slope " + str(slope))
        mean_price = np.mean(df_ntl_ppg[df_ntl_cust.uch_level_g == c].avg_price_per_unit_to_consmer_log)
        print("mean_price " + str(mean_price))
        mean_quantity = np.mean(df_ntl_ppg[df_ntl_cust.uch_level_g == c].total_sales_units_log)
        print("mean_quantity " + str(mean_quantity))
        tintercept, t_score = model_ntl_ppg.tvalues
        print("tintercept " + str(tintercept))
        print("t_score " + str(t_score))
        
        price_elasticity = (slope)
        
        #Append results into dictionary for dataframe
        ntl_ppg_results_values["ppg_id"].append(c)
        ntl_ppg_results_values["price_elasticity"].append(price_elasticity)
        ntl_ppg_results_values["price_mean"].append(mean_price)
        ntl_ppg_results_values["quantity_mean"].append(mean_quantity)
        ntl_ppg_results_values["intercept"].append(intercept)
        ntl_ppg_results_values['t_score'].append(t_score)
        ntl_ppg_results_values["slope"].append(slope)
        ntl_ppg_results_values["coefficient_pvalue"].append(coefficient_pvalue)
        ntl_ppg_results_values['rsquared'].append(rsquared)
        if coefficient_pvalue < 0.05:
            ntl_ppg_results_values["msg"].append("coefficient_pvalue < 0.05, statistically significant results")
        else:
            ntl_ppg_results_values["msg"].append("coefficient_pvalue >= 0.05, statistically insignificant results")
        print("Model results printed")
        print("*******************************************************************")
    except:
        print("Error occurred")
        ntl_err_list.append(c)

final_df_ntl_ppg = pd.DataFrame.from_dict(ntl_ppg_results_values)
df_ntl_ppg_elasticity = final_df_ntl_ppg[['ppg_id','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared','msg']]
pe_plot = divergent_plot(df_ntl_ppg_elasticity, 'price_elasticity', 'overall_ped_ranking', 'price_elasticity')

df_ntl_ppg_elasticity_l1 = pd.merge(df_ntl_ppg_elasticity, df_ppg_list, on = ['ppg_id'], how='inner')
ntl_ppg_elasticity_conditions = [
    (df_ntl_ppg_elasticity_l1['price_elasticity'] <= 0) & (df_ntl_ppg_elasticity_l1['price_elasticity'] > -1),
    (df_ntl_ppg_elasticity_l1['price_elasticity'] == -1.000000000),
    (df_ntl_ppg_elasticity_l1['price_elasticity'] < -1),
    (df_ntl_ppg_elasticity_l1['price_elasticity'] > 0)
    ]   
df_ntl_ppg_elasticity_l1['elasticity_explanation'] = np.select(ntl_ppg_elasticity_conditions, elasticity_explanation)
df_ntl_ppg_elasticity_l1.to_csv("canada_national_ppg_elasticity_details.csv", index=False)
df_ntl_ppg_elasticity_l1[['ppg_id', 'ppg_desc','price_elasticity', 'overall_ped_ranking', 'msg', 'elasticity_explanation']].sort_values(by = ['price_elasticity']).to_csv("canada_national_customer_elasticity_summary.csv", index=False)

In [ ]:
df_ntl_ppg

In [ ]:
df_ppg

In [ ]:
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
'''
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
df = pd.read_csv("result_elasticity_data_ctg.csv")
df.columns = [c.lower() for c in df.columns]
df['total_sales_units_log'] = np.log(df.total_sales_units)
df['avg_price_per_unit_to_consmer_log'] = np.log(df.avg_price_per_unit_to_consmer)
orig_df = df.copy()
#df = df[df.prod_category.isin(br1_ctg)]
df = df.sort_values(by=['fscl_wk_end_dt', 'prod_category', 'uch_level_g'])

profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile
'''

'\nimport pandas as pd\nimport numpy as np\nfrom pandas_profiling import ProfileReport\ndf = pd.read_csv("result_elasticity_data_ctg.csv")\ndf.columns = [c.lower() for c in df.columns]\ndf[\'total_sales_units_log\'] = np.log(df.total_sales_units)\ndf[\'avg_price_per_unit_to_consmer_log\'] = np.log(df.avg_price_per_unit_to_consmer)\norig_df = df.copy()\n#df = df[df.prod_category.isin(br1_ctg)]\ndf = df.sort_values(by=[\'fscl_wk_end_dt\', \'prod_category\', \'uch_level_g\'])\n\nprofile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)\n'

default


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/basic.py:300: UserWarning: %profile is now deprecated. Please use get_ipython().profile instead.
  warn("%profile is now deprecated. Please use get_ipython().profile instead.")
